In [ ]:
import os
import anthropic

from langchain_anthropic.chat_models import ChatAnthropic
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# Chat 모델 초기화
chat_model = ChatAnthropic(model="claude-3-opus-20240229", temperature=0.1)
template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}"
)

template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a cryptocurrency market expert. And you only reply in {language}.",
        ),
        ("ai", "안녕하세요!"),
        ("human", "Do you know {something}?"),
    ]
)

prompt = template.format(language="Greek", something="pendle's v3 release day")

chat_model.predict_messages(prompt)

In [16]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        item = text.strip().split(",")
        return list(map(str.strip, item))  # 공백도 제거


# p=CommaOutputParser()
# p.parse("Hello, How,are,u,?")

In [17]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are a list generating machine. Everything you are asked"
                " wull be answered with a comma seperted list of max"
                " {max_items}. Do not reply with anything else."
            ),
        ),
        ("human", "{question}"),
    ]
)

prompt = template.format(max_items=10, question="what are the colors?")

In [ ]:
from langchain_anthropic.chat_models import ChatAnthropic
from langchain.callbacks import StreamingStdOutCallbackHandler
chat_model = ChatAnthropic(model="claude-3-opus-20240229", temperature=0.1, streaming = True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        (
            "You are a world-class international chef. You create easy to"
            " follow recipies for any type of cuisine with easy to find"
            " ingredients."
        ),
    ),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat_model

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."
            ),
        ),
        ("human", "{recipe}"),
    ]
)

veg_chain = veg_chef_prompt | chat_model
final_chain = {"recipe":chef_chain} | veg_chain 

result = final_chain.invoke({
  "cuisine" :"indian"
})

print(result)